# White House visitor logs

### Import Python tools and Jupyter configuration

In [33]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [35]:
import altair as alt
import altair_latimes as lat
import numpy as np

In [36]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [37]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Data dictionary

In [47]:
# "https://obamawhitehouse.archives.gov/files/disclosures/visitors/WhiteHouse-WAVES-Key-1209.txt"

In [38]:
today = dt.date.today().strftime("%m-%d-%Y")

In [39]:
df = pd.read_csv(
    "data/processed/log_latest.csv",
    parse_dates=[
        "appt_made_date",
        "appt_start_date",
        "appt_end_date",
        "appt_cancel_date",
        "releasedate",
    ],
)

In [49]:
df.sort_values(["total_people"], ascending=False).head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
198,MCBRIDE,JEREMY,V,U19761,NaN,VA,NAN,NAN,NAN,NAN,2021-01-16 10:06:00,2021-01-20 13:00:00,2021-01-20 23:59:00,NaT,34.0,DV,WIN,1/16/2021 10:06,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,HERALD TRUMPETS,2021-05-07
90,EISENSTADT,ALICIA,M,U19761,NaN,VA,NAN,NAN,NAN,NAN,2021-01-16 10:06:00,2021-01-20 13:00:00,2021-01-20 23:59:00,NaT,34.0,DV,WIN,1/16/2021 10:06,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,HERALD TRUMPETS,2021-05-07
266,POWLISON,JOHN,M,U19761,NaN,VA,NAN,NAN,NAN,NAN,2021-01-16 10:06:00,2021-01-20 13:00:00,2021-01-20 23:59:00,NaT,34.0,DV,WIN,1/16/2021 10:06,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,HERALD TRUMPETS,2021-05-07
332,STOCKTON,TRENT,A,U19761,NaN,VA,NAN,NAN,NAN,NAN,2021-01-16 10:06:00,2021-01-20 13:00:00,2021-01-20 23:59:00,NaT,34.0,DV,WIN,1/16/2021 10:06,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,HERALD TRUMPETS,2021-05-07
140,HOOKE,CHARLES,M,U19761,NaN,VA,NAN,NAN,NAN,NAN,2021-01-16 10:06:00,2021-01-20 13:00:00,2021-01-20 23:59:00,NaT,34.0,DV,WIN,1/16/2021 10:06,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,HERALD TRUMPETS,2021-05-07


### Buildings

In [41]:
df.meeting_loc.value_counts()

EEOB    221
WH      138
NEOB     41
Name: meeting_loc, dtype: int64

### Rooms 

In [42]:
df.meeting_room.value_counts()

EW206         89
EEOB 210/2    50
94            23
3202          15
594           15
430           14
3201          14
210           13
180           12
RES CLINIC    11
97             7
EAST EXEC/     7
INDIAN TRE     6
415            6
208            6
OVAL OFFIC     6
97-WHMU CL     6
PASS OFFIC     5
1B29           5
474            4
STATE FLOO     4
291            4
76             4
97-WHMU ME     4
4000           4
1              3
96-WHMU ME     3
300            3
SITUATION      2
WW G67         2
LAFAYETTE      2
WEST EXEC      2
EW TOUR        2
1ST FLOOR      2
18             2
386            2
71             2
LOADING DO     2
NEOB LOADI     2
21             2
79             2
212            2
USHERS OF      2
14             1
490            1
330            1
312            1
316            1
432            1
183            1
RESIDENCE      1
17             1
422            1
MEDICAL        1
WW G66         1
CLINIC         1
RM 22          1
526            1
EW 121        

### Who's visited the Oval?

In [43]:
df[df["meeting_room"].str.contains("OVAL OFFIC")]

In [50]:
df[df["meeting_room"].str.contains("EW206")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
9,BAILEY,JANET,C,U19818,NaN,VA,NAN,NAN,NAN,NAN,2021-01-18 09:14:00,2021-01-20 12:00:00,2021-01-20 23:59:00,NaT,19.0,DV,WIN,1/18/2021 9:14,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,MARINE BAND SUPPORTING EVENT,2021-05-07
13,BARR,ROGER,B,U19950,166794.0,VA,1/21/2021 10:03,A0401,1/21/2021 11:38,A0401,2021-01-21 09:27:00,2021-01-21 09:30:00,2021-01-21 23:59:00,NaT,10.0,ET,WIN,1/21/2021 9:27,ET,TELEKY,ED,WH,EW206,TELEKY,EDWARD,NaN,NAN,2021-05-07
14,BARR,ROGER,B,U19919,NaN,VA,NAN,NAN,NAN,NAN,2021-01-20 19:04:00,2021-01-20 20:00:00,2021-01-20 23:59:00,NaT,10.0,ET,WIN,1/20/2021 19:04,ET,TELEKY,ED,WH,EW206,TELEKY,EDWARD,NaN,NAN,2021-05-07
15,BARR,ROGER,B,U19959,NaN,VA,NAN,NAN,NAN,NAN,2021-01-21 10:08:00,2021-01-21 10:45:00,2021-01-21 23:59:00,NaT,10.0,ET,WIN,1/21/2021 10:08,ET,TELEKY,ED,WH,EW206,TELEKY,EDWARD,NaN,NAN,2021-05-07
24,BISHOP,JAMES,M,U19761,NaN,VA,NAN,NAN,NAN,NAN,2021-01-16 10:06:00,2021-01-20 13:00:00,2021-01-20 23:59:00,NaT,34.0,DV,WIN,1/16/2021 10:06,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,HERALD TRUMPETS,2021-05-07


### Where 'visitee' is 'POTUS'

In [57]:
df[df["visitee_namefirst"].str.contains("")]

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
